---
description: Drop-in replacement of OpenAI SDK to get full observability in Langfuse by changing only the import
category: Integrations
---

# Cookbook: OpenAI Integration (Python)

This is a cookbook with examples of the Langfuse Integration for OpenAI (Python).

Follow the [integration guide](https://langfuse.com/docs/integrations/openai/get-started) to add this integration to your OpenAI project.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## Setup

The integration is compatible with OpenAI SDK versions `>=0.27.8`. It supports async functions and streaming for OpenAI SDK versions `>=1.0.0`.

In [2]:
%pip install langfuse openai --upgrade


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# instead of: import openai
from langfuse.openai import openai

In [4]:
# For debugging, checks the SDK connection with the server. Do not use in production as it adds latency.
from langfuse.openai import auth_check

auth_check()

True

## Examples

### Chat completion

In [5]:
completion = openai.chat.completions.create(
  name="test-chat",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 1 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)

### Chat completion (streaming)

Simple example using the OpenAI streaming functionality.

In [6]:
completion = openai.chat.completions.create(
  name="test-chat",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a professional comedian."},
      {"role": "user", "content": "Tell me a joke."}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end="")

Sure thing! Why did the scarecrow win an award? Because he was outstanding in his field!None

### Chat completion (async)

Simple example using the OpenAI async client. It takes the Langfuse configurations either from the environment variables or from the attributes on the `openai` module.

In [7]:
from langfuse.openai import AsyncOpenAI

async_client = AsyncOpenAI()

In [8]:
completion = await async_client.chat.completions.create(
  name="test-chat",
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": "You are a very accurate calculator. You output only the result of the calculation."},
      {"role": "user", "content": "1 + 100 = "}],
  temperature=0,
  metadata={"someMetadataKey": "someValue"},
)

Go to https://cloud.langfuse.com or your own instance to see your generation.

![Chat completion](https://langfuse.com/images/docs/openai-chat.png)

### Functions

Simple example using Pydantic to generate the function schema.

In [9]:
%pip install pydantic --upgrade

  Using cached pydantic-2.6.4-py3-none-any.whl (394 kB)
  Using cached pydantic_core-2.16.3-cp310-cp310-macosx_11_0_arm64.whl (1.7 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.10.0
    Uninstalling pydantic_core-2.10.0:
      Successfully uninstalled pydantic_core-2.10.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.4.0
    Uninstalling pydantic-2.4.0:
      Successfully uninstalled pydantic-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlmodel 0 requires pydantic[email]<=2.4,>=2.1.1, but you have pydantic 2.6.4 which is incompatible.
langflow 0.6.0a0 requires cohere<5.0.0,>=4.32.0, but you have cohere 5.1.2 which is incompatible.
langflow 0.6.0a0 requires langchain<0.1.0,>=0.0.327, but you have langchain 0.1.13 which is incompatible.
langflow 0.6.0a0 requires langfuse<2.0.0,>

In [10]:
from typing import List
from pydantic import BaseModel

class StepByStepAIResponse(BaseModel):
    title: str
    steps: List[str]
schema = StepByStepAIResponse.schema() # returns a dict like JSON schema

/var/folders/m3/cz7sy7cd4v7_mklwrfjs45wm0000gn/T/ipykernel_99748/2496491748.py:7: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  schema = StepByStepAIResponse.schema() # returns a dict like JSON schema


In [11]:
import json
response = openai.chat.completions.create(
    name="test-function",
    model="gpt-3.5-turbo-0613",
    messages=[
       {"role": "user", "content": "Explain how to assemble a PC"}
    ],
    functions=[
        {
          "name": "get_answer_for_user_query",
          "description": "Get user answer in series of steps",
          "parameters": StepByStepAIResponse.schema()
        }
    ],
    function_call={"name": "get_answer_for_user_query"}
)

output = json.loads(response.choices[0].message.function_call.arguments)

/var/folders/m3/cz7sy7cd4v7_mklwrfjs45wm0000gn/T/ipykernel_99748/162860978.py:12: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  "parameters": StepByStepAIResponse.schema()


Go to https://cloud.langfuse.com or your own instance to see your generation.

![Function](https://langfuse.com/images/docs/openai-function.png)


### Group multiple generations into a single trace

Many applications require more than one OpenAI call. The `@observe()` decorator allows to nest all LLM calls of a single API invocation into the same `trace` in Langfuse.

In [12]:
from langfuse.openai import openai
from langfuse.decorators import observe

@observe() # decorator to automatically create trace and nest generations
def main(country: str, user_id: str, **kwargs) -> str:
    # nested generation 1: use openai to get capital of country
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked."},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content

    # nested generation 2: use openai to write poem on capital
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a poet. Create a poem about a city."},
          {"role": "user", "content": capital}],
      temperature=1,
      max_tokens=200,
    ).choices[0].message.content

    return poem

# run main function and let Langfuse decorator do the rest
print(main("Bulgaria", "admin"))

In the heart of the Balkans, she stands so proud,
A city of shadows and light, both silent and loud.
With ancient streets that weave and wind,
Through history's tapestry, a story to find.

Sofia, the city of red-roofed homes,
Where saints and sinners freely roam.
The Alexander Nevsky Cathedral so grand,
A beacon of faith in this ancient land.

Among the bustling markets and café's cheer,
The spirit of Sofia is ever near.
From Vitosha Mountain, her guardian high,
To the Serdika ruins where old worlds lie.

The heartbeat of Bulgaria, a city so alive,
In her cobblestone streets, stories thrive.
With each passing moment, a new tale begun,
Sofia, eternal, beneath the Balkan sun.

Her people, vibrant, diverse and strong,
In unity and resilience, they belong.
A city of contrasts, old and new,
Sofia, forever in my heart, I'll hold


Go to https://cloud.langfuse.com or your own instance to see your trace.

![Trace with multiple OpenAI calls](https://langfuse.com/images/docs/openai-trace-grouped.png)

#### Fully featured: Interoperability with Langfuse SDK

The `trace` is a core object in Langfuse and you can add rich metadata to it. See [Python SDK docs](https://langfuse.com/docs/sdk/python#traces-1) for full documentation on this.

Some of the functionality enabled by custom traces:
- custom name to identify a specific trace-type
- user-level tracking
- experiment tracking via versions and releases
- custom metadata

In [16]:
import uuid
from langfuse.openai import openai
from langfuse.decorators import langfuse_context, observe

@observe() # decorator to automatically create trace and nest generations
def main(country: str, user_id: str, **kwargs) -> str:
    # nested generation 1: use openai to get capital of country
    capital = openai.chat.completions.create(
      name="geography-teacher",
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked."},
          {"role": "user", "content": country}],
      temperature=0,
    ).choices[0].message.content

    # nested generation 2: use openai to write poem on capital
    poem = openai.chat.completions.create(
      name="poet",
      model="gpt-3.5-turbo",
      messages=[
          {"role": "system", "content": "You are a poet. Create a poem about a city."},
          {"role": "user", "content": capital}],
      temperature=1,
      max_tokens=200,
    ).choices[0].message.content

    # rename trace and set attributes (e.g., medatata) as needed
    langfuse_context.update_current_trace(
        name="City poem generator",
        session_id="1234",
        user_id=user_id,
        tags=["tag1", "tag2"],
        public=True,
        metadata = {
        "env": "development",
        },
        release = "v0.0.21"
    )

    return poem

# create random trace_id, could also use existing id from your application, e.g. conversation id
trace_id = str(uuid.uuid4())

# run main function, set your own id, and let Langfuse decorator do the rest
print(main("Bulgaria", "admin", langfuse_observation_id=trace_id))

In the heart of the Balkans, where history meets modernity,
Lies a city of beauty, Sofia, a place of pure serenity.
With ancient ruins whispering tales of days long gone,
And vibrant street art that dances with the dawn.

A melting pot of cultures, where East meets West,
Sofia's charm will put your wandering soul to the test.
The rhythm of the city pulses through its veins,
As vibrant markets bustle and soothing fountains reign.

Beneath the shadow of Vitosha, the mountain so grand,
Sofia stands proud, a jewel in Bulgaria's hand.
With its grand cathedrals and majestic domes,
It's a city that calls you to wander and roam.

From the bustling boulevards to quiet cobbled lanes,
Sofia's spirit will stir in your heart like gentle rains.
So come, wanderer, and let the city reveal,
The magic and wonder that its streets conceal.


### Programmatically add scores

You can add [scores](https://langfuse.com/docs/scores) to the trace, to e.g. record user feedback or some programmatic evaluation. Scores are used throughout Langfuse to filter traces and on the dashboard. See the docs on scores for more details.

The score is associated to the trace using the `trace_id`.

In [17]:
from langfuse import Langfuse
from langfuse.decorators import langfuse_context, observe

langfuse = Langfuse()

@observe() # decorator to automatically create trace and nest generations
def main():
    # get trace_id of current trace
    trace_id = langfuse_context.get_current_trace_id()

    # rest of your application ...

    return "res", trace_id

# execute the main function to generate a trace
_, trace_id = main()

# Score the trace from outside the trace context
langfuse.score(
    trace_id=trace_id,
    name="my-score-name",
    value=1
)